In [329]:
import keras, numpy as np, matplotlib.pyplot as plt
from keras.datasets import mnist

(x_train,y_train),(x_test,y_test) = mnist.load_data()

x_train = x_train.astype('float')/255
x_test = x_test.astype('float')/255

x_train = np.reshape(x_train,[60000,784])
y_train = np.reshape(y_train,[60000,1])

x_test = np.reshape(x_test,[10000,784])
y_test = np.reshape(y_test,[10000,1])

z = np.zeros((60000,10))
for i in range(60000):
    z[i][y_train[i]] = 1
y_train = z

z = np.zeros((10000,10))
for i in range(10000):
    z[i][y_test[i]] = 1
y_test = z

del z

In [330]:
np.random.seed(0)
w1 = np.random.randn(784,512) * 0.2
np.random.seed(1)
w2 = np.random.randn(512,256) * 0.2
np.random.seed(2)
w3 = np.random.randn(256,128) * 0.2
np.random.seed(3)
w4 = np.random.randn(128,64) * 0.2
np.random.seed(4)
w5 = np.random.randn(64,10) * 0.2

b1 = np.zeros((1,512))
b2 = np.zeros((1,256))
b3 = np.zeros((1,128))
b4 = np.zeros((1,64))
b5 = np.zeros((1,10))

In [331]:
def normalize(x):
    num = (x - np.min(x,axis=1).reshape(len(x),1))
    den = (np.max(x,axis=1).reshape(len(x),1) - np.min(x,axis=1).reshape(len(x),1))
    return num/den

In [332]:
def softmax(x):
    X = normalize(x)
    e_x = np.exp(X) 
    soft = e_x / (np.sum(e_x,axis=1).reshape(len(x),1))
    return soft

In [333]:
def tanh(x):
    X = normalize(x)
    t = (np.exp(X)-np.exp(-X)) / (np.exp(X)+np.exp(-X))
    return t

In [334]:
def dtanh(x):
    t=tanh(x)
    dt=1-t**2
    return dt

In [335]:
def relu(X):
    x = normalize(X)
    return x * (x > 0)

In [336]:
def drelu(X):
    x = normalize(X)
    return 1. * (x > 0)

In [337]:
def update(x,y,w1,b1,w2,b2,w3,b3,w4,b4,w5,b5,lr):
    
    z1 = np.dot(x,w1) + b1
    a1 = np.array(relu(z1))
    
    z2 = np.dot(a1,w2) + b2
    a2 = np.array(relu(z2))
    
    z3 = np.dot(a2,w3) + b3
    a3 = np.array(relu(z3))

    z4 = np.dot(a3,w4) + b4
    a4 = np.array(relu(z4))
    
    z5 = np.dot(a4,w5) + b5
    a5 = np.array(softmax(z5))
    
    dz5 = a5 - y
    dw5 = (1/len(x)) * np.dot(a4.T,dz5)
    db5 = (1/len(x)) * np.sum(dz5,axis=0,keepdims=True)
    
    dz4 = np.dot(dz5,w5.T) * drelu(z4)
    dw4 = (1/len(x)) * np.dot(a3.T,dz4)
    db4 = (1/len(x)) * np.sum(dz4,axis=0,keepdims=True)
    
    dz3 = np.dot(dz4,w4.T) * drelu(z3)
    dw3 = (1/len(x)) * np.dot(a2.T,dz3)
    db3 = (1/len(x)) * np.sum(dz3,axis=0,keepdims=True)
    
    dz2 = np.dot(dz3,w3.T) * drelu(z2)
    dw2 = (1/len(x)) * np.dot(a1.T,dz2)
    db2 = (1/len(x)) * np.sum(dz2,axis=0,keepdims=True)
    
    dz1 = np.dot(dz2,w2.T) * drelu(z1)
    dw1 = (1/len(x)) * np.dot(x.T,dz1)
    db1 = (1/len(x)) * np.sum(dz1,axis=0,keepdims=True)
    
    w1 -= lr*dw1
    b1 -= lr*db1
    w2 -= lr*dw2
    b2 -= lr*db2
    w3 -= lr*dw3
    b3 -= lr*db3
    w4 -= lr*dw4
    b4 -= lr*db4
    w5 -= lr*dw5
    b5 -= lr*db5
    
    return [w1,b1,w2,b2,w3,b3,w4,b4,w5,b5]

In [338]:
for j in range(600):
    x_new = np.array([x_train[i] for i in range((100*j),(100*(j+1)))]).reshape(100,784)
    y_new = np.array([y_train[i] for i in range((100*j),(100*(j+1)))]).reshape(100,10)
    [w1,b1,w2,b2,w3,b3,w4,b4,w5,b5] = update(x_new,y_new,w1,b1,w2,b2,w3,b3,w4,b4,w5,b5,0.00099)

In [341]:
zt1 = np.dot(x_test,w1) + b1
at1 = np.array(relu(zt1))

zt2 = np.dot(at1,w2) + b2
at2 = np.array(relu(zt2))

zt3 = np.dot(at2,w3) + b3
at3 = np.array(relu(zt3))

zt4 = np.dot(at3,w4) + b4
at4 = np.array(relu(zt4))
    
zt5 = np.dot(at4,w5) + b5
at5 = np.array(softmax(zt5))

preds = np.argmax(at5,axis=1).reshape(len(at5),1)
truth = np.argmax(y_test,axis=1).reshape(len(y_test),1)

c = np.equal(preds,truth)
num = np.squeeze(np.sum(c,axis=0))
den = len(truth)
acc = (num/den) * 100
print(acc,"\b%")

63.800000000000004 %
